### Install the latest .whl package

Check [here](https://pypi.org/project/semantic-link-labs/) to see the latest version.

In [ ]:
%pip uninstall "builtin/semantic_link_labs-0.9.4-py3-none-any.whl" -y

In [ ]:
# %pip install semantic-link-labs

%pip install "builtin/semantic_link_labs-0.9.4-py3-none-any.whl"

### Requirements
* Fabric Capacity with XMLA read/write enabled
    * A Fabric Trial Capacity is sufficient for evaluation.
    * The XMLA Endpoint must be read/write enabled because the perf lab provisions semantic models automatically.
* Fabric Permissions
    * User must have permissions to create workspaces, lakehouses, and semantic models. This notebook provisions sample resources to demonstrate the use of a perf lab.
    * User should have access to a Fabric capacity. This notebook provisions workspaces, lakehouses, and semantic models on a Fabric capacity.
    * Connect this notebook to a lakehouse without a schema to persist test definitions and test results. Although strictly not a requirement, it eliminates the need to provide the name and Id of a disconnected lakehouse.

### Result
* A master and test workspaces, lakehouses, and semantic models are created to establish a perf lab
    * The master workspace contains a lakehouse and a sample semantic model in Direct Lake on OneLake mode that uses the lakehouse as its data source. 
    * The test workspace contains semantic models cloned from the sample semantic model in the master workspace.
    * Various Delta tables are created in the lakehouse connected to this notebook to persist test definitions, table analysis, and test results.
    * The resources in the master workspace and in the test workspace are deprovisioned upon completion of the perf lab. Delete the workspaces manually.
* The names of the newly created resources can be adjusted to customize the perf lab.


### Import the library and set global notebook parameters

This notebook deploys lakehouses and semantic models across different workspaces, but the resources can also be hosted together in a centralized workspace. The master workspace contains a lakehouse with sample data, used as the data source for the sample semantic models in Direct Lake on OneLake mode. The master semantic model serves as a template for the actual test models, which this notebook provisions prior to running the performance tests by cloning the master semantic model.

In [ ]:

import sempy_labs.perf_lab as perf_lab

master_workspace = 'Perf Lab Master'                # Enter the name of the master workspace.
lakehouse = 'SampleLakehouse'                       # Enter the name of the lakehouse used as the data source.
master_dataset = 'Master Semantic Model'            # Enter the name of the master semantic model.

test_workspace = 'Perf Lab Testing'                 # Enter the name of the workspace for the semantic model clone.
target_dataset_prefix = 'Test Model_'               # Enter the common part of the name for all semantic model clones.
test_dataset_A = target_dataset_prefix + 'A'        # Enter the name of the first semantic model clone.
test_dataset_B = target_dataset_prefix + 'B'        # Enter the name of the second semantic model clone.

capacity_id = None                                  # The Id of the capacity for the workspaces. 
                                                    # Leave this as None to use the capacity of the attached lakehouse or perf lab notebook.
                                        
test_definitions_tbl = 'TestDefinitions'            # The name of the table in the notebook-attached lakehouse to store the test definitions.
column_segments_tbl = 'StorageTableColumnSegments'  # The name of the table in the notebook-attached lakehouse to store the test definitions.
trace_events_tbl = "TraceEvents"                    # The name of the table in the notebook-attached lakehouse to store the captured trace events.

### Working with test definitions

Test definitions define the key parameters for the test runs, including the following fields: QueryId, QueryText, MasterWorkspace, MasterDataset, TargetWorkspace, TargetDataset, DatasourceName, DatasourceWorkspace, DatasourceType. The following test code illustrates how to work with the TestDefinition and TestSuite classes.


In [ ]:
first_test_definition = perf_lab.TestDefinition(QueryId=1, QueryText="Evaluate {1}", MasterWorkspace=master_workspace, MasterDataset=master_dataset,
                   TargetWorkspace=test_workspace, TargetDataset='Test Model_', DatasourceName=lakehouse,
                   DatasourceWorkspace=master_workspace, DatasourceType="WrongType")

first_test_definition.remove("DatasourceType")
print(first_test_definition.get_keys())
first_test_definition.add("DatasourceType", "Lakehouse")
first_test_definition.TargetDataset='Test Model_A'
print(first_test_definition.get_keys())
print(first_test_definition.get_values())
print(first_test_definition.to_schema())

test_definitions = [
    first_test_definition,
    perf_lab.TestDefinition(QueryId=2, QueryText="Evaluate {2}", MasterWorkspace=master_workspace, MasterDataset=master_dataset,
                   TargetWorkspace=test_workspace, TargetDataset='Test Model_B', DatasourceName=lakehouse,
                   DatasourceWorkspace=master_workspace, DatasourceType="Lakehouse")
]

test_suite = perf_lab.TestSuite(test_definitions)
test_suite.save_as(test_definitions_tbl)
display(test_suite.to_df())

test_suite.remove_test_definition(first_test_definition)
display(test_suite.to_df())

test_suite.clear()
test_suite.load(test_definitions_tbl)
display(test_suite.to_df())

test_suite.add_test_definition(
    perf_lab.TestDefinition(QueryId=3, QueryText="Evaluate {3}", MasterWorkspace=master_workspace, MasterDataset=master_dataset,
                   TargetWorkspace=test_workspace, TargetDataset='Test Model_C', DatasourceName=lakehouse,
                   DatasourceWorkspace=master_workspace, DatasourceType="Lakehouse")
)
display(test_suite.to_df())

test_suite.add_field("AdditionalProperty", "additional value")
print(test_suite.get_schema())

test_suite.remove_field("AdditionalProperty")
print(test_suite.get_schema())

test_suite.clear()
display(test_suite.to_df())


### Run default (incremental), cold, and warm query tests
The main purpose of a test run is to measure the performance of a set of DAX queries against the test semantic models with different memory states: Cold (full framing), Semi-warm (incremental framing), and Warm (no framing). Other than running the queries and measuring response times, the run_test_cycle() function must therefore perform additional actions, specifically clearing the cache and refreshing the model.

In [ ]:
import uuid
from datetime import datetime

test_workspace = master_workspace
test_dataset_A = "PartitionedSampleMasterModel"
test_suite = perf_lab.TestSuite(
    [
    perf_lab.TestDefinition(QueryId="TestQuery", QueryText="EVALUATE SUMMARIZECOLUMNS(\"Test\", \"Hello World\")", MasterWorkspace=master_workspace, MasterDataset=master_dataset,
                   TargetWorkspace=test_workspace, TargetDataset=test_dataset_A, DatasourceName=lakehouse,
                   DatasourceWorkspace=master_workspace, DatasourceType="Lakehouse", TestRunId = str(uuid.uuid4()), TestRunTimestamp = str(datetime.now()))
    ]
)

inc_results = perf_lab.run_test_cycle(
    test_suite = test_suite,
    clear_query_cache = True,
    refresh_type = "clearValuesFull",
    tag = "for test purposes"
    )

display(inc_results[0])
display(inc_results[1])